In [1]:
import random
import glob
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer,BertForSequenceClassification
import pytorch_lightning as pl

#日本語の事前学習モデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

C:\Users\kinar\.conda\envs\natural_langugage_processing\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#全記事の文章データを取得して前処理

#カテゴリーのリスト
category_list = [
    'api',
    'ec',
    'movie',
    'sns',
    'webrtc'
]

#トークナイザのロード
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)


max_length = 128#各テキストの最大長を128トークンに設定
dataset_for_loader = []#空のリストを初期化して、後でトークン化されたデータを格納

#category_listに含まれる各カテゴリについて、インデックス(ラベル)とカテゴリ名を取得
for label,category in enumerate(tqdm(category_list)):
    #指定したパターンに一致するファイルのリストを取得し、それぞれのファイルを処理
    for file in glob.glob(f'./text/{category}/{category}*'):
        #ファイルを開いて、すべての行を読み込み
        lines = open(file,encoding='utf-8').read().splitlines()
        #ファイルの最初の3行を除いた残りの内容を改行で連結
        #text = '\n' .join(lines[3:])
        text = '\n' .join(lines)
        #BERTトークナイザーを使用してテキストをトークン化
        encoding = tokenizer(
            text,
            max_length=max_length,
            padding='max_length',
            truncation=True
        )
        #トークン化されたデータにカテゴリのラベルを追加
        encoding['labels'] = label
        #トークナイザーの出力をPyTorchのテンソルに変換
        encoding = {k:torch.tensor(v) for k, v in encoding.items()}
        #前処理とラベル付けを完了したデータをリストに追加
        dataset_for_loader.append(encoding)
        
        # トークンをテキストにデコードし、ラベルと一緒に表示
        '''
        tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
        print(f"Sample Text Tokens: {tokens[:10]}")  # 最初の10トークンを表示
        print(f"Label: {label}")
        '''

100%|██████████| 5/5 [00:00<00:00, 41.50it/s]


In [3]:
#データセットをランダムにシャッフルし、サンプルの順番をランダム化
random.shuffle(dataset_for_loader)
#データセットの全体の長さ（サンプル数）を取得
n = len(dataset_for_loader)
#データセットの60%を訓練データ
n_train = int(0.6*n)
#データセットの20%を検証データ
n_val = int(0.2*n)
#計算された割合に基づいて、データセットを訓練、検証、テストデータに分割
dataset_train = dataset_for_loader[:n_train]#学習データ
dataset_val = dataset_for_loader[n_train:n_train+n_val]#検証データ
dataset_test = dataset_for_loader[n_train+n_val:]#テストデータ

#訓練用のデータを作成、バッチサイズの指定を行い、各エポックごとでデータの順序をランダムにする。
dataloader_train = DataLoader(
    dataset_train,batch_size=32,shuffle=True
)
#検証用とテスト用のデーターローダを作成
datasloader_val = DataLoader(dataset_val,batch_size=64)
dataloader_test = DataLoader(dataset_test,batch_size=64)

In [4]:
#文章分類モデルの定義
from transformers import BertForSequenceClassification, BertConfig
class BertSewuenceClassification_pl(pl.LightningModule):
    
    def __init__(self,model_name,num_labels,lr):
        '''
        引数として、モデル名、ラベルの数、学習率を受け取る
        '''
        super().__init__()
        #後からモデルのハイパーパラメータを確認
        self.save_hyperparameters()
        
        #BertForSequenceClassificationモデルの初期化
        '''
        model_name：使用するBERTの事前学習済みモデルの名前
        num_class：分類タスクのクラス数
        '''
        config = BertConfig.from_pretrained(model_name, num_labels=num_labels)
        self.bert_sc = BertForSequenceClassification.from_pretrained(model_name, config=config)
    
    #モデルの順伝播を定義
    def forward(self, input_ids, attention_mask=None, labels=None):
        output = self.bert_sc(input_ids, attention_mask=attention_mask, labels=labels)
        return output
    
    #訓練時の各バッチに対する処理を定義
    def training_step(self,batch,batch_idx):
        '''
        batch：データローダーから取得される1バッチ分のデータ
        batch_idx：現在のバッチ番号
        '''
        #バッチ内のモデルの出力
        output = self.bert_sc(**batch)
        #バッチ内の損失値
        loss = output.loss
        self.log('train_loss',loss)
        return loss
    
    #検証時の各バッチに対する処理を定義します。ここでも損失を計算し、ログに記録
    def validation_step(self,batch,batch_idx):
        '''
        batch：データローダーから取得される1バッチ分のデータ
        batch_idx：現在のバッチ番号
        '''
        output = self.bert_sc(**batch)
        val_loss = output.loss
        self.log('val_loss',val_loss)
        
    def test_step(self,batch,batch_idx):
        '''
        batch：データローダーから取得される1バッチ分のデータ
        batch_idx：現在のバッチ番号
        '''
        #テストデータのバッチ全体を表す辞書から正解ラベルを取り出す
        labels = batch.pop('labels')
        #事前に定義された分類タスク用のBERTモデルにbatchを渡し、出力を取得
        output = self.bert_sc(**batch)
        #最もスコアが高いクラスのインデックスを取得
        labels_predicted = output.logits.argmax(-1)
        #予測されたクラスと正解ラベルを比較し、一致するかどうかをブール値で取得
        num_correct = (labels_predicted == labels).sum().item()
        #精度を計算
        accuracy = num_correct/labels.size(0)
        self.log('accuracy',accuracy)
        
    #モデルの最適化手法を定義、Adamオプティマイザを使用し、コンストラクタで設定された学習率を適用
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(),lr=self.hparams.lr)

In [5]:
#トレーニング中に特定の条件（モニタリングするメトリクス）を基にモデルを保存
checkpoint = pl.callbacks.ModelCheckpoint(
    #モニタリングするメトリクスとして、バリデーション損失を指定
    monitor='val_loss',
    #モニタリングするメトリクスの最小値を基準にモデルを保存
    mode='min',
    #最良のモデル（val_lossが最小のモデル）1つのみを保存
    save_top_k=1,
    #モデルの重み（パラメータ）のみを保存
    save_weights_only=True,
    #保存先ディレクトリを指定
    dirpath='model/',
)

from pytorch_lightning.callbacks import EarlyStopping

# 早期停止の設定
early_stop_callback = EarlyStopping(
    monitor='val_loss',   # 監視する値
    min_delta=0.00,       # 改善とみなされる最小の変化
    patience=2,           # 改善が見られないエポック数
    verbose=True,         # ログを出力
    mode='min'            # 「min」は監視値の減少を目指す（損失の場合）
)


#Trainer クラスを使用して訓練のプロセスを設定
trainer = pl.Trainer(
    #使用するGPUの数を指定
    accelerator="gpu",  # GPUを使用
    devices=1, # 使用するGPUの数 
    max_epochs=10,# 最大エポック数
    #トレーニング中に実行するコールバック（追加処理）のリストを指定
    callbacks=[checkpoint, early_stop_callback]
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [6]:
model = BertSewuenceClassification_pl(
    MODEL_NAME,num_labels=5,lr=1e-5
)

#トレーニング開始
trainer.fit(model,dataloader_train,datasloader_val)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                          | Params | Mode
-----------------------------------------------------------------
0 | bert_sc | BertForSequenceClassification | 110 M  | eval
-----------------------------------------------------------------
110 M     Train

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\kinar\.conda\envs\natural_langugage_processing\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


C:\Users\kinar\.conda\envs\natural_langugage_processing\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
C:\Users\kinar\.conda\envs\natural_langugage_processing\Lib\site-packages\pytorch_lightning\loops\fit_loop.py:298: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0: 100%|██████████| 10/10 [00:02<00:00,  3.87it/s, v_num=0]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 0: 100%|██████████| 10/10 [00:02<00:00,  3.41it/s, v_num=0]     

...

Epoch 8: 100%|██████████| 10/10 [00:02<00:00,  4.07it/s, v_num=0]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 8: 100%|██████████| 10/10 [00:02<00:00,  3.53it/s, v_num=0]     

Metric val_loss improved by 0.030 >= min_delta = 0.0. New best score: 0.440


Epoch 9: 100%|██████████| 10/10 [00:02<00:00,  4.07it/s, v_num=0]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 9: 100%|██████████| 10/10 [00:02<00:00,  3.52it/s, v_num=0]     

Metric val_loss improved by 0.024 >= min_delta = 0.0. New best score: 0.415
`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 10/10 [00:03<00:00,  2.56it/s, v_num=0]


In [7]:
best_model_path = checkpoint.best_model_path#ベストモデルのファイル
print('ベストモデルのファイル:',checkpoint.best_model_path)
print('ベストモデルの検証データに対する損失:',checkpoint.best_model_score)

ベストモデルのファイル: C:\Users\kinar\Desktop\data-preparation\model\epoch=9-step=100.ckpt
ベストモデルの検証データに対する損失: tensor(0.4153, device='cuda:0')


In [8]:
# テストデータでの評価
test_result = trainer.test(model, dataloaders=dataloader_test)
print(f"Accuracy on test data: {test_result[0]['accuracy']:.2f}")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\kinar\.conda\envs\natural_langugage_processing\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  6.25it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.9100000262260437     │
└───────────────────────────┴───────────────────────────┘

Accuracy on test data: 0.91


In [21]:
#PyTorch Lightningモデルのロード
model = BertSewuenceClassification_pl.load_from_checkpoint(
    best_model_path
)

#Transformers対応モデルを./model_transformersに保存
model.bert_sc.save_pretrained('./model_transformers')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
bert_sc = BertForSequenceClassification.from_pretrained(
    './model_transformers/'
)

In [25]:
bert_sc.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [30]:
# 分析したいテキスト
text = "テキストで、コミュニケーションを行えるサービスを開発したいです。"

# テキストをトークナイズしてテンソルに変換
inputs = tokenizer(text, return_tensors="pt")

In [31]:
import torch

# 推論を実行（勾配計算は不要）
with torch.no_grad():
    outputs = bert_sc(**inputs)

# ロジットから予測されたクラスIDを取得
predicted_class_id = outputs.logits.argmax(-1).item()
print(f"Predicted class ID: {predicted_class_id}")

Predicted class ID: 3


NoSuchFile: [ONNXRuntimeError] : 3 : NO_SUCHFILE : Load model from bert_model.onnx failed:Load model bert_model.onnx failed. File doesn't exist